# 06: Human-in-the-Loop with Interrupts

Use LangGraph's `interrupt` function for proper human-in-the-loop control.

In [13]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

from typing import Annotated
from langchain_core.tools import tool
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.types import Command, interrupt
from typing_extensions import TypedDict

from config.llm_factory import LLMFactory

print("✅ Ready for human-in-the-loop with interrupts!")

✅ Ready for human-in-the-loop with interrupts!


## Define State and Tools

In [14]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

@tool
def verify_claim(claim: str) -> str:
    """Verify an aviation claim against known facts."""
    facts = {
        "747 engines": "The Boeing 747 has 4 engines",
        "wright brothers": "First flight was December 17, 1903",
        "concorde speed": "Concorde flew at Mach 2.04"
    }
    
    claim_lower = claim.lower()
    for key, fact in facts.items():
        if key in claim_lower:
            return fact
    return "No specific data found for this claim."

@tool 
def human_review(claim: str, ai_verdict: str, confidence: int) -> str:
    """Request human review when confidence is low."""
    # Interrupt execution and wait for human input
    human_response = interrupt({
        "claim": claim,
        "ai_verdict": ai_verdict, 
        "confidence": confidence,
        "reason": f"Low confidence ({confidence}%) - need human verification"
    })
    return human_response["verdict"]

tools = [verify_claim, human_review]

## Build the Graph

In [15]:
# Create LLM with tools
llm = LLMFactory.create_llm()
llm_with_tools = llm.bind_tools(tools)

def bs_detector(state: State):
    """Main BS detection node"""
    # Add system message if this is the first message
    messages = state["messages"]
    if len(messages) == 1:  # Only user message
        system_msg = {
            "role": "system",
            "content": """You are an aviation BS detector. 
            First use verify_claim to check facts.
            Then determine if the claim is BS or LEGITIMATE with a confidence score.
            If your confidence is below 60%, use human_review for help."""
        }
        messages = [system_msg] + messages
    
    response = llm_with_tools.invoke(messages)
    # Limit to one tool call at a time for clarity
    assert len(response.tool_calls) <= 1
    return {"messages": [response]}

# Build graph
graph_builder = StateGraph(State)

# Add nodes
graph_builder.add_node("bs_detector", bs_detector)
tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

# Add edges
graph_builder.add_edge(START, "bs_detector")
graph_builder.add_conditional_edges(
    "bs_detector",
    tools_condition,
)
graph_builder.add_edge("tools", "bs_detector")

# Compile with checkpointer for persistence
memory = InMemorySaver()
graph = graph_builder.compile(checkpointer=memory)

## Test High Confidence Claim (No Interrupt)

In [16]:
# High confidence claim - should not trigger human review
config = {"configurable": {"thread_id": "1"}}

events = graph.stream(
    {"messages": [{"role": "user", "content": "The Boeing 747 has 4 engines. Is this BS?"}]},
    config,
    stream_mode="values"
)

for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

The Boeing 747 has 4 engines. Is this BS?
================================== Ai Message ==================================
Tool Calls:
  verify_claim (call_6alvBEtBFQPsDQTvvm1ENEuq)
 Call ID: call_6alvBEtBFQPsDQTvvm1ENEuq
  Args:
    claim: The Boeing 747 has 4 engines.
================================= Tool Message =================================
Name: verify_claim

No specific data found for this claim.
================================== Ai Message ==================================
Tool Calls:
  human_review (call_sgfNB8aSz1ue9U1ZZ4dOxYhX)
 Call ID: call_sgfNB8aSz1ue9U1ZZ4dOxYhX
  Args:
    claim: The Boeing 747 has 4 engines.
    ai_verdict: Unknown
    confidence: 50


## Test Low Confidence Claim (With Interrupt)

In [17]:
# Low confidence claim - should trigger human review
config2 = {"configurable": {"thread_id": "2"}}

print("🤖 Testing uncertain claim...\n")

events = graph.stream(
    {"messages": [{"role": "user", "content": "A new quantum jet can fly at light speed. Is this BS?"}]},
    config2,
    stream_mode="values"
)

for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

🤖 Testing uncertain claim...

================================ Human Message =================================

A new quantum jet can fly at light speed. Is this BS?
================================== Ai Message ==================================
Tool Calls:
  verify_claim (call_x2ME9CzoucmuSLhwdDmJMAan)
 Call ID: call_x2ME9CzoucmuSLhwdDmJMAan
  Args:
    claim: A new quantum jet can fly at light speed.
================================= Tool Message =================================
Name: verify_claim

No specific data found for this claim.
================================== Ai Message ==================================
Tool Calls:
  human_review (call_hJ2ADevSm2gY7UPmseINHrKd)
 Call ID: call_hJ2ADevSm2gY7UPmseINHrKd
  Args:
    claim: A new quantum jet can fly at light speed.
    ai_verdict: No specific data found for this claim.
    confidence: 40


## Check Interrupted State

In [18]:
# Check where execution stopped
snapshot = graph.get_state(config2)
print(f"\n🛑 Execution interrupted at: {snapshot.next}")

# Get the interrupt data
if snapshot.next:
    print("\n📋 Interrupt details:")
    last_message = snapshot.values["messages"][-1]
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        for tool_call in last_message.tool_calls:
            if tool_call['name'] == 'human_review':
                print(f"Claim: {tool_call['args']['claim']}")
                print(f"AI Verdict: {tool_call['args']['ai_verdict']}")
                print(f"Confidence: {tool_call['args']['confidence']}%")


🛑 Execution interrupted at: ('tools',)

📋 Interrupt details:
Claim: A new quantum jet can fly at light speed.
AI Verdict: No specific data found for this claim.
Confidence: 40%


## Resume with Human Input

In [19]:
# Provide human verdict
print("\n👤 HUMAN REVIEW")
print("The claim about quantum jets flying at light speed is definitely BS.\n")

# Resume execution with human input
human_command = Command(
    resume={"verdict": "DEFINITELY BS - Violates laws of physics. Nothing with mass can reach light speed."}
)

events = graph.stream(human_command, config2, stream_mode="values")
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()


👤 HUMAN REVIEW
The claim about quantum jets flying at light speed is definitely BS.

================================== Ai Message ==================================
Tool Calls:
  human_review (call_hJ2ADevSm2gY7UPmseINHrKd)
 Call ID: call_hJ2ADevSm2gY7UPmseINHrKd
  Args:
    claim: A new quantum jet can fly at light speed.
    ai_verdict: No specific data found for this claim.
    confidence: 40
================================= Tool Message =================================
Name: human_review

DEFINITELY BS - Violates laws of physics. Nothing with mass can reach light speed.
================================== Ai Message ==================================

The claim that a new quantum jet can fly at light speed is definitely not true. It violates the fundamental laws of physics, as nothing with mass can reach the speed of light. This claim is BS.


## Interactive Session

In [ ]:
def run_interactive_bs_detector():
    """Run an interactive BS detection session"""
    thread_id = 100
    
    print("🎮 Interactive BS Detector (type 'quit' to exit)\n")
    
    while True:
        claim = input("\nEnter a claim to check: ")
        if claim.lower() == 'quit':
            break
            
        thread_id += 1
        config = {"configurable": {"thread_id": str(thread_id)}}
        
        # Stream the response
        events = graph.stream(
            {"messages": [{"role": "user", "content": f"Is this claim BS: {claim}"}]},
            config,
            stream_mode="values"
        )
        
        for event in events:
            if "messages" in event:
                event["messages"][-1].pretty_print()
        
        # Check if interrupted
        snapshot = graph.get_state(config)
        if snapshot.next:
            print("\n⚠️  Human review needed!")
            verdict = input("Your verdict (BS/LEGITIMATE): ")
            explanation = input("Brief explanation: ")
            
            # Resume with human input
            human_command = Command(
                resume={"verdict": f"{verdict} - Human: {explanation}"}
            )
            
            events = graph.stream(human_command, config, stream_mode="values")
            for event in events:
                if "messages" in event:
                    event["messages"][-1].pretty_print()

# Uncomment to run
run_interactive_bs_detector()

🎮 Interactive BS Detector (type 'quit' to exit)




Enter a claim to check:  Boeing was better than Airbus in avaition safety in 2024.


================================ Human Message =================================

Is this claim BS: Boeing was better than Airbus in avaition safety in 2024.
================================== Ai Message ==================================
Tool Calls:
  verify_claim (call_LjpaaqTPl1ZKhzLl0Dvdpvv1)
 Call ID: call_LjpaaqTPl1ZKhzLl0Dvdpvv1
  Args:
    claim: Boeing was better than Airbus in aviation safety in 2024.
================================= Tool Message =================================
Name: verify_claim

No specific data found for this claim.
================================== Ai Message ==================================
Tool Calls:
  human_review (call_kNuHD0QX4Gm3TdRwv9oHsRRt)
 Call ID: call_kNuHD0QX4Gm3TdRwv9oHsRRt
  Args:
    claim: Boeing was better than Airbus in aviation safety in 2024.
    ai_verdict: No specific data found to verify this claim.
    confidence: 40

⚠️  Human review needed!


Your verdict (BS/LEGITIMATE):  Naah. Too many flights has system issues. Not reliable. 
Brief explanation:  Lot of crashes with 737 and dreamliner planes across the world. 


================================== Ai Message ==================================
Tool Calls:
  human_review (call_kNuHD0QX4Gm3TdRwv9oHsRRt)
 Call ID: call_kNuHD0QX4Gm3TdRwv9oHsRRt
  Args:
    claim: Boeing was better than Airbus in aviation safety in 2024.
    ai_verdict: No specific data found to verify this claim.
    confidence: 40
================================= Tool Message =================================
Name: human_review

Naah. Too many flights has system issues. Not reliable.  - Human: Lot of crashes with 737 and dreamliner planes across the world. 
================================== Ai Message ==================================

The claim that Boeing was better than Airbus in aviation safety in 2024 cannot be confirmed based on available data. In fact, there have been notable safety concerns and incidents involving Boeing aircraft such as the 737 and Dreamliner models. These issues have affected the perception of Boeing's safety performance. Therefore, the claim appears t

## Key Concepts

1. **`interrupt()` function**: Pauses execution and waits for human input
2. **`Command` object**: Used to resume execution with data
3. **Checkpointer**: Enables persistence across interrupts
4. **Tool-based approach**: Human review is just another tool

This approach allows:
- Indefinite pausing (as long as checkpointer persists)
- Clean separation between AI and human decisions
- Full audit trail of interactions
- Easy integration with UIs (web apps, etc.)